In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

#/content/drive/MyDrive/combined_precipitation_array.npy
#/content/drive/MyDrive/combined_urbanization_images.npy
#/content/drive/MyDrive/crop_yield_images/combined_crop_yield_images.npy

precipitation_array = np.load('/content/drive/MyDrive/Urbanization Project/arrays of graphs/combined_precipitation_array.npy')
urbanization_array = np.load('/content/drive/MyDrive/Urbanization Project/arrays of graphs/combined_urbanization_images.npy')
crop_yield_array = np.load('/content/drive/MyDrive/Urbanization Project/arrays of graphs/combined_crop_yield_images.npy')
temperature_array = np.load('/content/drive/MyDrive/Urbanization Project/arrays of graphs/temperature_final_array.npy')

In [ ]:
print(precipitation_array.shape)
print(urbanization_array.shape)
print(crop_yield_array.shape)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, LSTM, Reshape, TimeDistributed, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import set_global_policy

# Set mixed precision
set_global_policy('mixed_float16')

# Ensure TensorFlow is using GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is available")
else:
    print("No GPU available, using CPU")

def build_smaller_hybrid_model(input_shape=(23, 32, 32, 3), lstm_units=16, unet_filters=16):
    inputs = Input(shape=input_shape)

    # Temporal processing with LSTM
    x = TimeDistributed(Flatten())(inputs)
    x = LSTM(lstm_units, return_sequences=False)(x)
    x = Dense(32 * 32 * lstm_units)(x)
    x = Reshape((32, 32, lstm_units))(x)

    # Simplified U-Net
    conv1 = Conv2D(unet_filters, 3, activation='relu', padding='same')(x)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(unet_filters*2, 3, activation='relu', padding='same')(pool1)

    up3 = UpSampling2D(size=(2, 2))(conv2)
    up3 = concatenate([up3, conv1])
    conv3 = Conv2D(unet_filters, 3, activation='relu', padding='same')(up3)

    outputs = Conv2D(3, 1, activation='linear')(conv3)

    model = Model(inputs=inputs, outputs=outputs)
    return model

def resize_image(img, new_size):
    return tf.image.resize(img, new_size).numpy()

def preprocess_data(precipitation_array, urbanization_array, crop_yield_array, temperature_array, new_size=(32, 32)):
    def resize_array_images(arr):
        shape = arr.shape
        resized = np.zeros((shape[0], shape[1], shape[2], new_size[0], new_size[1], 3))
        for i in range(shape[0]):
            for j in range(shape[1]):
                for k in range(shape[2]):
                    resized[i,j,k] = resize_image(arr[i,j,k], new_size)
        return resized

    precipitation_array = resize_array_images(precipitation_array)
    urbanization_array = resize_array_images(urbanization_array)
    crop_yield_array = resize_array_images(crop_yield_array)
    temperature_array = resize_array_images(temperature_array)

    return precipitation_array, urbanization_array, crop_yield_array, temperature_array

def data_generator(precipitation_array, urbanization_array, crop_yield_array, temperature_array, batch_size=32, split='train'):
    years, lat_partitions, lon_partitions, height, width, channels = precipitation_array.shape
    train_years = int(0.8 * years)

    while True:
        if split == 'train':
            year_range = range(5, train_years)
        else:
            year_range = range(train_years, years)

        for year in year_range:
            for lat in range(0, lat_partitions, batch_size):
                for lon in range(0, lon_partitions, batch_size):
                    X_batch = []
                    y_batch = []

                    for i in range(lat, min(lat + batch_size, lat_partitions)):
                        for j in range(lon, min(lon + batch_size, lon_partitions)):
                            x_sample = []

                            # Last 5 years of all data
                            for past_year in range(year-5, year):
                                x_sample.append(crop_yield_array[past_year, i, j])
                                x_sample.append(temperature_array[past_year, i, j])
                                x_sample.append(precipitation_array[past_year, i, j])
                                x_sample.append(urbanization_array[past_year, i, j])

                            # Current year's temp, precip, urban
                            x_sample.append(temperature_array[year, i, j])
                            x_sample.append(precipitation_array[year, i, j])
                            x_sample.append(urbanization_array[year, i, j])

                            X_batch.append(np.array(x_sample))
                            y_batch.append(crop_yield_array[year, i, j])

                    if len(X_batch) == batch_size:
                        yield np.array(X_batch), np.array(y_batch)
                        X_batch = []
                        y_batch = []

        # Yield any remaining samples
        if X_batch:
            yield np.array(X_batch), np.array(y_batch)

# Main execution
if __name__ == "__main__":
    # Load data
    precipitation_array = np.load('/content/drive/MyDrive/combined_precipitation_array.npy')
    urbanization_array = np.load('/content/drive/MyDrive/combined_urbanization_images.npy')
    crop_yield_array = np.load('/content/drive/MyDrive/crop_yield_images/combined_crop_yield_images.npy')
    temperature_array = np.load('/content/drive/MyDrive/Copy of temperature_final_array.npy')

    # Preprocess data
    new_size = (32, 32)
    precipitation_array, urbanization_array, crop_yield_array, temperature_array = preprocess_data(
        precipitation_array, urbanization_array, crop_yield_array, temperature_array, new_size)

    # Create the generators
    batch_size = 32  # Set batch size
    train_gen = data_generator(precipitation_array, urbanization_array, crop_yield_array, temperature_array, batch_size=batch_size, split='train')
    val_gen = data_generator(precipitation_array, urbanization_array, crop_yield_array, temperature_array, batch_size=batch_size, split='val')

    # Build and compile the model
    input_shape = (23, 32, 32, 3)  # 23 time steps, 32x32 image, 3 channels
    model = build_smaller_hybrid_model(input_shape)

    optimizer = Adam(learning_rate=0.001)  # Increased learning rate
    model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae', tf.keras.metrics.RootMeanSquaredError()])
    # Calculate steps per epoch
    years, lat_partitions, lon_partitions, _, _, _ = precipitation_array.shape
    train_samples = (int(0.8 * years) - 5) * lat_partitions * lon_partitions
    steps_per_epoch = train_samples // batch_size
    validation_steps = (years - int(0.8 * years)) * lat_partitions * lon_partitions // batch_size

    # Train the model
    history = model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=9,
                        validation_data=val_gen, validation_steps=validation_steps)

    # Print final metrics
    print(f"Final Training MAE: {history.history['mae'][-1]:.4f}")
    print(f"Final Validation MAE: {history.history['val_mae'][-1]:.4f}")

    # Save the model
    model.save('smaller_hybrid_model.keras')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Bidirectional, Conv2D, MaxPooling2D, UpSampling2D, concatenate, LSTM, Reshape, TimeDistributed, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import set_global_policy

# Set mixed precision
set_global_policy('mixed_float16')

# Ensure TensorFlow is using GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is available")
else:
    print("No GPU available, using CPU")

def build_smaller_hybrid_model(input_shape=(23, 32, 32, 3), lstm_units=16, unet_filters=16):
    inputs = Input(shape=input_shape)

    # Temporal processing with LSTM
    x = TimeDistributed(Flatten())(inputs)
    x = Bidirectional(LSTM(lstm_units, return_sequences=False))(x)
    x = Dense(32 * 32 * lstm_units)(x)
    x = Reshape((32, 32, lstm_units))(x)

    # Simplified U-Net
    conv1 = Conv2D(unet_filters, 3, activation='relu', padding='same')(x)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(unet_filters*2, 3, activation='relu', padding='same')(pool1)

    up3 = UpSampling2D(size=(2, 2))(conv2)
    up3 = concatenate([up3, conv1])
    conv3 = Conv2D(unet_filters, 3, activation='relu', padding='same')(up3)

    outputs = Conv2D(3, 1, activation='linear')(conv3)

    model = Model(inputs=inputs, outputs=outputs)
    return model

    # Simplified U-Net
    conv1 = Conv2D(unet_filters, 3, activation='relu', padding='same')(x)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(unet_filters*2, 3, activation='relu', padding='same')(pool1)

    up3 = UpSampling2D(size=(2, 2))(conv2)
    up3 = concatenate([up3, conv1])
    conv3 = Conv2D(unet_filters, 3, activation='relu', padding='same')(up3)

    outputs = Conv2D(3, 1, activation='linear')(conv3)

    model = Model(inputs=inputs, outputs=outputs)
    return model

def resize_image(img, new_size):
    return tf.image.resize(img, new_size).numpy()

def preprocess_data(precipitation_array, urbanization_array, crop_yield_array, temperature_array, new_size=(32, 32)):
    def resize_array_images(arr):
        shape = arr.shape
        resized = np.zeros((shape[0], shape[1], shape[2], new_size[0], new_size[1], 3))
        for i in range(shape[0]):
            for j in range(shape[1]):
                for k in range(shape[2]):
                    resized[i,j,k] = resize_image(arr[i,j,k], new_size)
        return resized

    precipitation_array = resize_array_images(precipitation_array)
    urbanization_array = resize_array_images(urbanization_array)
    crop_yield_array = resize_array_images(crop_yield_array)
    temperature_array = resize_array_images(temperature_array)

    return precipitation_array, urbanization_array, crop_yield_array, temperature_array

def data_generator(precipitation_array, urbanization_array, crop_yield_array, temperature_array, batch_size=32, split='train'):
    years, lat_partitions, lon_partitions, height, width, channels = precipitation_array.shape
    train_years = int(0.8 * years)

    while True:
        if split == 'train':
            year_range = range(5, train_years)
        else:
            year_range = range(train_years, years)

        for year in year_range:
            for lat in range(0, lat_partitions, batch_size):
                for lon in range(0, lon_partitions, batch_size):
                    X_batch = []
                    y_batch = []

                    for i in range(lat, min(lat + batch_size, lat_partitions)):
                        for j in range(lon, min(lon + batch_size, lon_partitions)):
                            x_sample = []

                            # Last 5 years of all data
                            for past_year in range(year-5, year):
                                x_sample.append(crop_yield_array[past_year, i, j])
                                x_sample.append(temperature_array[past_year, i, j])
                                x_sample.append(precipitation_array[past_year, i, j])
                                x_sample.append(urbanization_array[past_year, i, j])

                            # Current year's temp, precip, urban
                            x_sample.append(temperature_array[year, i, j])
                            x_sample.append(precipitation_array[year, i, j])
                            x_sample.append(urbanization_array[year, i, j])

                            X_batch.append(np.array(x_sample))
                            y_batch.append(crop_yield_array[year, i, j])

                    if len(X_batch) == batch_size:
                        yield np.array(X_batch), np.array(y_batch)
                        X_batch = []
                        y_batch = []

        # Yield any remaining samples
        if X_batch:
            yield np.array(X_batch), np.array(y_batch)

# Main execution
if __name__ == "__main__":
    # Load data
    precipitation_array = np.load('/content/drive/MyDrive/Urbanization Project/arrays of graphs/combined_precipitation_array.npy')
    urbanization_array = np.load('/content/drive/MyDrive/Urbanization Project/arrays of graphs/combined_urbanization_images.npy')
    crop_yield_array = np.load('/content/drive/MyDrive/Urbanization Project/arrays of graphs/combined_crop_yield_images.npy')
    temperature_array = np.load('/content/drive/MyDrive/Urbanization Project/arrays of graphs/temperature_final_array.npy')

    # Preprocess data
    new_size = (32, 32)
    precipitation_array, urbanization_array, crop_yield_array, temperature_array = preprocess_data(
        precipitation_array, urbanization_array, crop_yield_array, temperature_array, new_size)

    # Create the generators
    batch_size = 32  # Set batch size
    train_gen = data_generator(precipitation_array, urbanization_array, crop_yield_array, temperature_array, batch_size=batch_size, split='train')
    val_gen = data_generator(precipitation_array, urbanization_array, crop_yield_array, temperature_array, batch_size=batch_size, split='val')

    # Build and compile the model
    input_shape = (23, 32, 32, 3)  # 23 time steps, 32x32 image, 3 channels
    model = build_smaller_hybrid_model(input_shape)

    optimizer = Adam(learning_rate=0.001)  # Increased learning rate
    model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae', tf.keras.metrics.RootMeanSquaredError()])
    # Calculate steps per epoch
    years, lat_partitions, lon_partitions, _, _, _ = precipitation_array.shape
    train_samples = (int(0.8 * years) - 5) * lat_partitions * lon_partitions
    steps_per_epoch = train_samples // batch_size
    validation_steps = (years - int(0.8 * years)) * lat_partitions * lon_partitions // batch_size

    # Train the model
    history = model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=9,
                        validation_data=val_gen, validation_steps=validation_steps)

    # Print final metrics
    print(f"Final Training MAE: {history.history['mae'][-1]:.4f}")
    print(f"Final Validation MAE: {history.history['val_mae'][-1]:.4f}")

    # Save the model
    model.save('UNet_hybrid_model.keras')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')